In [3]:
import matplotlib.pyplot as plt
import numpy as np
	
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkComplexity2/raw/master/notebooks/utils.py')
download('https://github.com/AllenDowney/ThinkComplexity2/raw/master/notebooks/Cell2D.py')

In [5]:
from utils import decorate, savefig
!mkdir -p figs

In [7]:
try:
    import empiricaldist
except ImportError:
    !pip install empiricaldist  

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for empiricaldist: filename=empiricaldist-0.7.4-py3-none-any.whl size=12278 sha256=cf08da4c43b6b4ee652c5d3e8a4375c0741891dc161c4411f257cead061c7999
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\99\9d\27\3dc07cda48f486cb3c5938a18ddc47fd50023852841c3c3e42
Successfully built empiricaldist


In [9]:
def locs_where(condition):
    return list(zip(*np.nonzero(condition)))

In [11]:
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
	
palette = sns.color_palette('muted')
colors = 'white', palette[1], palette[0]
cmap = LinearSegmentedColormap.from_list('cmap', colors)  

In [13]:
from scipy.signal import correlate2d
from Cell2D import Cell2D, draw_array
	
class Schelling(Cell2D):
    options = dict(mode='same', boundary='wrap')
		
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]], dtype=np.int8)
						   
    def __init__(self, n, p):
        self.p = p
        choices = np.array([0, 1, 2], dtype=np.int8)
        probs = [0.1, 0.45, 0.45]
        self.array = np.random.choice(choices, (n, n), p=probs)

In [15]:
    def count_neighbors(self):
        a = self.array
			
        empty = a==0
        red = a==1
        blue = a==2
			
        num_red = correlate2d(red, self.kernel, **self.options)
        num_blue = correlate2d(blue, self.kernel, **self.options)
        num_neighbors = num_red + num_blue
			
        frac_red = num_red / num_neighbors
        frac_blue = num_blue / num_neighbors
			
        frac_red[num_neighbors == 0] = 0
        frac_blue[num_neighbors == 0] = 0
			
        frac_same = np.where(red, frac_red, frac_blue)
        frac_same[empty] = np.nan
			
        return empty, frac_red, frac_blue, frac_same

